# DMPlex from pyGMsh with labels

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pygmsh
import numpy as np

In [2]:
resolution=0.1

## Create a simple unstructured rectangular mesh

In [3]:
L = 1.0
H = 1.0

In [4]:
geom = pygmsh.geo.Geometry()

In [5]:
# Fetch model we would like to add data to
model = geom.__enter__()

In [6]:
points = [geom.add_point((0.,0.,0.), mesh_size=resolution),
          geom.add_point((L,0.,0.), mesh_size=resolution),
          geom.add_point((L,H,0.), mesh_size=resolution),
          geom.add_point((0,H,0.), mesh_size=resolution)]

Add lines between all points creating the rectangle

In [7]:
lines = [geom.add_line(points[i], points[i+1]) for i in range(-1, len(points)-1)]

Create a line loop and plane surface for meshing

In [8]:
loop = geom.add_curve_loop(lines)
surface = geom.add_plane_surface(loop)

In [9]:
geom.synchronize()

In [10]:
left, bottom, right, top = lines

In [11]:
left_marker = left.dim_tag[1]
right_marker = right.dim_tag[1]
top_marker = top.dim_tag[1]
bottom_marker = bottom.dim_tag[1]

In [12]:
geom.add_physical([left], "Left")
geom.add_physical([right], "Right")
geom.add_physical([top], "Top")
geom.add_physical([bottom], "Bottom")

In [13]:
mesh = geom.generate_mesh(dim=2)

In [14]:
import gmsh
gmsh.option.set_number("Mesh.SaveAll", 1)
gmsh.write("mesh_test.msh")
gmsh.clear()
geom.__exit__()

# Import Mesh into PETSc using DMPlex

In [15]:
import sys,petsc4py
petsc4py.init(sys.argv)
from petsc4py import PETSc
import numpy as np

In [16]:
options = PETSc.Options()

In [17]:
options["dm_plex_separate_marker"] = None

In [18]:
plex = PETSc.DMPlex().createFromFile("mesh_test.msh")

In [19]:
plex.view()

DM Object: DM_0x556bacfaed20_0 1 MPI processes
  type: plex
DM_0x556bacfaed20_0 in 2 dimensions:
  0-cells: 142
  1-cells: 383
  2-cells: 242
Labels:
  celltype: 3 strata with value/size (0 (142), 3 (242), 1 (383))
  depth: 3 strata with value/size (0 (142), 1 (383), 2 (242))
  Face Sets: 4 strata with value/size (1 (10), 4 (10), 2 (10), 3 (10))


In [20]:
markers_dict = {"Bottom": bottom_marker,
                "Right": right_marker,
                "Top": top_marker,
                "Left": left_marker}

In [21]:
markers_dict

{'Bottom': 2, 'Right': 3, 'Top': 4, 'Left': 1}

In [22]:
for key, value in markers_dict.items():
    indexSet = plex.getStratumIS("Face Sets", value)
    plex.createLabel(key)
    label = plex.getLabel(key)
    if indexSet:
        label.insertIS(indexSet, value)
    indexSet.destroy()

In [23]:
plex.view()

DM Object: DM_0x556bacfaed20_0 1 MPI processes
  type: plex
DM_0x556bacfaed20_0 in 2 dimensions:
  0-cells: 142
  1-cells: 383
  2-cells: 242
Labels:
  celltype: 3 strata with value/size (0 (142), 3 (242), 1 (383))
  depth: 3 strata with value/size (0 (142), 1 (383), 2 (242))
  Face Sets: 4 strata with value/size (1 (10), 4 (10), 2 (10), 3 (10))
  Bottom: 1 strata with value/size (2 (10))
  Right: 1 strata with value/size (3 (10))
  Top: 1 strata with value/size (4 (10))
  Left: 1 strata with value/size (1 (10))
